In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import OLSInfluence
import os
import pickle

/tmp/ipykernel_1383335/2519447265.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
wetchem_df = pd.read_csv("/home/tom/DSML125/inputFiles/uncleaned_wetchem.csv")

In [3]:
len(wetchem_df.index.unique())

13997

In [4]:
wetchem_df.columns

Index(['Unnamed: 0', 'sample_code', 'ph', 'calcium'], dtype='object')

In [5]:
def cooks_distance_outliers(column): 
#     x = wetchem_df.dropna(subset=[column])[column].values
#     samples = wetchem_df.dropna(subset=[column])["sample_code"].values
# #     x.sort()
#     y = []
#     count = 0
#     for i in x:
#         y.append(count)
#         count+=1
#     f = 'x ~ y'
#     df = pd.DataFrame({"x":x, "y":y, "sample_code": samples})
#     model = ols(formula=f, data=df).fit()

#     cook_distance = OLSInfluence(model).cooks_distance
#     (distance, p_value) = cook_distance
#     df['distance'] = distance
#     mean_cooks_dist = df["distance"].mean()
    
    
#     outliers = df.loc[df["distance"] > 3*(mean_cooks_dist)]
#     outliers = outliers.rename(columns={"x": column})
#     outliers.drop('y', axis=1,inplace=True)
#     outliers.drop('distance', axis=1,inplace=True)
    
#     df_ = df.loc[df["distance"] <= 3*(mean_cooks_dist)]
#     df_ = df_.rename(columns={"x": column})
    
#     df.drop('y', axis=1,inplace=True)
#     df.drop('distance', axis=1,inplace=True)
    
#     if(column == 'exchangeable_acidity' or column == 'clay'):
#         _ = pd.DataFrame()
#         _[column] = []
        
#         ex = wetchem_df.dropna(subset=[column])
#         ex = ex[[column,'sample_code']]
#         ex= ex.reset_index()
#         print(ex.columns)
# #         print(ex['exchangeable_acidity'].describe())
#         return ex, _
    
    
#     vals = list(df_[column].values)

#     df_[column] = vals
                
    
    return df_, outliers

In [6]:
df_no_outliers = pd.DataFrame()
df_cooks_outliers = pd.DataFrame()

In [7]:
# cooks_distance_outliers("aluminium")

In [8]:
# for col in wetchem_df.columns:
#     if(col != 'sample_code'):
#         print(col)
#         no_cooks_outliers, cooks_outliers = cooks_distance_outliers(col)
#         df_no_outliers = pd.concat([df_no_outliers,no_cooks_outliers[['sample_code', col]]], axis=0)
#         df_cooks_outliers = pd.concat([df_cooks_outliers,cooks_outliers], axis=0)


In [9]:
# df_no_outliers = df_no_outliers.groupby("sample_code", sort=True).max()
# df_cooks_outliers = df_cooks_outliers.groupby("sample_code", sort=True).max()

In [10]:
# wetchem_df['exchangeable_acidity'].describe()

In [11]:
# df_no_outliers['exchangeable_acidity'].describe()

In [12]:
# wetchem_df['clay'].describe()

In [13]:
# df_no_outliers.clay.describe()

In [14]:
# df_cooks_outliers.loc[df_cooks_outliers.index == "CC003SA0047"]

In [15]:
# df_no_outliers.replace(0.0, np.nan, inplace=True)
# df_cooks_outliers.replace(0.0, np.nan, inplace=True)

In [16]:
# df_cooks_outliers

In [17]:
# len(df_no_outliers.index)

In [18]:
# len(df_cooks_outliers.index)

In [19]:
122009 + 21258

143267

In [20]:
# 
# len(wetchem_df['sample_code'].unique())

In [21]:
# df_cooks_outliers

In [22]:
# df_cooks_outliers.to_csv("outputFiles/outliers_cooks_distance.csv")
# df_no_outliers.to_csv("outputFiles/cleaned_data_cooks_distance.csv")

In [23]:
cooks_df_filtered = wetchem_df

In [24]:
# cooks_df_filtered['psi'].describe()

In [25]:
# wetchem_df = df_no_outliers

In [26]:
# wetchem_df.psi.describe()

In [27]:
# wetchem_df

In [28]:
high_value_filter_df = pd.DataFrame()
low_value_filter_df = pd.DataFrame()

In [29]:
all_chemicals = pd.read_csv("/home/tom/DSML125/inputFiles/modeling-instructions.csv")['chemical'].values

In [30]:
element_deviations = {}

In [31]:
print(element_deviations)

{}


In [32]:
def get_outliers(chemical):
    upper = wetchem_df[chemical].quantile(0.95)
    lower = wetchem_df[chemical].quantile(0.05)
    
    wetchem_data_copy = wetchem_df.copy()
    wetchem_data_copy.dropna(inplace=True, subset=[chemical])
    
    upper_outliers = wetchem_df[wetchem_df[chemical] > upper]
    lower_outliers = wetchem_df[wetchem_df[chemical] < lower]
    no_outliers = wetchem_df[(wetchem_df[chemical] >= lower) & (wetchem_df[chemical] <= upper)]
    
    return lower_outliers, no_outliers, upper_outliers

In [33]:
def get_gaps(upper_outliers, lower_outliers, column):
    gaps = []
    element_deviations[column] = {}
    for i in range(len(upper_outliers)-2):
        if(upper_outliers[i+1] - upper_outliers[i] > 0):
            gaps.append(upper_outliers[i+1] - upper_outliers[i])
    upper_quartile_gap = abs(np.quantile(gaps,0.25))
    element_deviations[column]["upper_quartile_deviation"] = upper_quartile_gap
    for i in range(len(lower_outliers)-2):
        if(lower_outliers[i+1] - lower_outliers[i] > 0):
            gaps.append(lower_outliers[i+1] - lower_outliers[i])
    lower_quartile_gap = abs(np.quantile(gaps,0.25))
    element_deviations[column]["lower_quartile_deviation"] = lower_quartile_gap
    return lower_quartile_gap, upper_quartile_gap

In [34]:
def _filter(outliers, gap_size, chemical, upper_or_lower):
    print(chemical)
    print(outliers[chemical])
    if(len(outliers) == 0):
        return wetchem_df[chemical]
    values = np.sort(outliers[chemical].values)
    print(gap_size)
    if(upper_or_lower == "lower"):
        values = values[::-1]
    count = 0
    for i,j in enumerate(values):
        if(i == len(values)-1):
            break
        if( abs(values[i] - values[i+1]) > gap_size  ):
            break
        count = i
    if(upper_or_lower == "lower"):
        print(upper_or_lower)
        cutoff = min(values[0:count])
        return outliers.loc[outliers[chemical] >= cutoff], outliers.loc[outliers[chemical] < cutoff]
    if(upper_or_lower == "upper"):
        print(upper_or_lower)
        cutoff = max(values[0:count])
        return outliers.loc[outliers[chemical] <= cutoff], outliers.loc[outliers[chemical] > cutoff]

In [35]:
wetchem_df

,Unnamed: 0,sample_code,ph,calcium
0,0,AGD103SA0012,6.28,4230.0
1,1,AGD103SA0018,6.06,4230.0
2,2,AGD207-408SA0842,4.99,882.0
3,3,AGD207-408SA0843,4.74,758.0
4,4,AGD207-408SA0844,4.57,815.0
...,...,...,...,...
13992,13992,I38-1-49SA0005,6.22,892.0
13993,13993,I38-1-50SA0001,5.67,110.0
13994,13994,I38-1-50SA0003,5.43,173.0
13995,13995,I38-1-50SA0004,5.62,88.3


In [36]:
high_val_filtered_df = pd.DataFrame()
high_val_outliers_df = pd.DataFrame()

low_val_filtered_df = pd.DataFrame()
low_val_outliers_df = pd.DataFrame()


In [37]:
high_val_filtered_df

""


In [38]:
for column in wetchem_df.columns:
    if(column != "sample_code" and (column in all_chemicals)):
        lower_outliers, no_outliers, upper_outliers = get_outliers(column)
        if len(lower_outliers) > 4: 
            lower_quartile_gap,  upper_quartile_gap = get_gaps(upper_outliers[column].values, upper_outliers[column].values, column)
            lv_filtered , lv_outliers = _filter(lower_outliers,element_deviations[column]["lower_quartile_deviation"], column, "lower")
            hv_filtered, hv_outliers = _filter(upper_outliers,element_deviations[column]["upper_quartile_deviation"],column, "upper")
            print("==========================")
        if len(upper_outliers) > 4: 
            high_val_filtered_df =   pd.concat([hv_filtered[['sample_code', column]], high_val_filtered_df])
            high_val_outliers_df = pd.concat([hv_outliers[['sample_code', column]], high_val_outliers_df])
            low_val_filtered_df = pd.concat([lv_filtered[['sample_code', column]], low_val_filtered_df])
            low_val_outliers_df = pd.concat([lv_outliers[['sample_code', column]], low_val_outliers_df])
            print("==========================")
        
    

ph
0.08999999999999986
ph
0.08999999999999986
calcium
350.0
calcium
350.0


In [39]:
high_val_filtered_df = high_val_filtered_df.groupby("sample_code").min()
high_val_outliers_df = high_val_outliers_df.groupby("sample_code").min()

low_val_filtered_df = low_val_filtered_df.groupby("sample_code").min()
low_val_outliers_df = low_val_outliers_df.groupby("sample_code").min()


In [40]:
high_val_filtered_df

,calcium,ph
sample_code,,
AGD207-408SA0872,6580.0,NaN
AGD207-408SA0880,NaN,7.99
AGD511SA0011,7010.0,NaN
AGD511SA0014,7210.0,NaN
AGD517-205SA0001,7400.0,NaN
...,...,...
FA907SA0017,6090.0,NaN
FA913SA0009,9190.0,NaN
FA913SA0010,7060.0,8.36


In [41]:
# high_val_filtered_df.psi.max()

In [42]:
# high_val_filtered_df = high_val_filtered_df.groupby("sample_code", sort=True).min()
# high_val_outliers_df = high_val_outliers_df.groupby("sample_code", sort=True).min()

# low_val_filtered_df = low_val_filtered_df.groupby("sample_code", sort=True).min()
# low_val_outliers_df = low_val_outliers_df.groupby("sample_code", sort=True).min()

# md_val_filtered_df = md_val_filtered_df.groupby("sample_code", sort=True).min()
# md_val_outliers_df = md_val_outliers_df.groupby("sample_code", sort=True).min()

In [43]:
# high_val_filtered_df.psi.max()

In [44]:
# high_val_filtered_df.replace(0,np.nan, inplace=True)
# high_val_outliers_df.replace(0,np.nan, inplace=True)

# low_val_filtered_df.replace(0,np.nan, inplace=True)
# low_val_outliers_df.replace(0,np.nan, inplace=True)

# md_val_filtered_df.replace(0,np.nan, inplace=True)
# md_val_outliers_df.replace(0,np.nan, inplace=True)

In [45]:
# high_val_filtered_df.psi.max()

In [46]:
high_val_filtered_df = high_val_filtered_df.apply(pd.to_numeric)
high_val_outliers_df = high_val_outliers_df.apply(pd.to_numeric)

low_val_filtered_df = low_val_filtered_df.apply(pd.to_numeric)
low_val_outliers_df = low_val_outliers_df.apply(pd.to_numeric)


In [47]:
high_val_outliers_df

,calcium,ph
sample_code,,
AGD517SA0030,18300.0,NaN
AGD518SA0001,NaN,9.30
AGD710-14SA0001,12500.0,NaN
AGD710-16SA0001,19000.0,NaN
AGD710-20SA0001,24800.0,NaN
...,...,...
FA534SA0007,NaN,10.20
FA539-63SA0001,13800.0,NaN
FA539SA0139,NaN,10.30


In [48]:
high_val_filtered_df.to_csv("/home/tom/DSML125/DSML87/outputFiles/hv_filtered.csv")
high_val_outliers_df.to_csv("/home/tom/DSML125/DSML87/outputFiles/hv_outliers.csv")

low_val_filtered_df.to_csv("/home/tom/DSML125/DSML87/outputFiles/lv_filtered.csv")
low_val_outliers_df.to_csv("/home/tom/DSML125/DSML87/outputFiles/lv_outliers.csv")



In [49]:
# import seaborn as sns
# for chemical in wetchem_df.columns:
#     sns.set_theme(style="whitegrid")
#     if(chemical in [x for x in wetchem_df.columns if x in all_chemicals]):
#         print(chemical)
#         fig, axes = plt.subplots(2, 3, figsize=(15, 5), sharey=True)
#         fig.suptitle(chemical)
        
#         lo = wetchem_df[chemical].min()
#         hi = wetchem_df[chemical].max()
#         print(lo)
#         print(hi)

#         sns.boxplot(ax=axes[0,0], x=wetchem_df[chemical])
#         axes[0,0].set_title(f"Uncleaned Wetchem Data n=0")
#         plt.clf()

#         plt.xlim(lo, hi)
#         sns.boxplot(ax=axes[0,2], x=high_val_filtered_df[chemical])
#         axes[0,2].set_title(f"High Gap Size Filter n={len(wetchem_df[chemical].dropna().values) - len(high_val_filtered_df[chemical].dropna().values)}")
#         plt.clf()
#         plt.xlim(lo, hi)
#         sns.boxplot(ax=axes[1,0], x=low_val_filtered_df[chemical])
#         axes[1,0].set_title(f"Low Gap Size Filter high_val_filtered_df n={len(wetchem_df[chemical].dropna().values) - len(low_val_filtered_df[chemical].dropna().values)}")
#         plt.clf()
#         plt.xlim(lo, hi)
#         sns.boxplot(ax=axes[1,1], x=md_val_filtered_df[chemical])
#         axes[1,1].set_title(f"Median Gap Size Outliers n={len(wetchem_df[chemical].dropna().values) - len(md_val_filtered_df[chemical].dropna().values)}")
#         plt.clf()
#         os.makedirs(f"/home/tom/DSML125/DSML87/outputFiles/boxplots", exist_ok=True)
#         plt.savefig(f"/home/tom/DSML125/DSML87/outputFiles/boxplots/{chemical}")